In [1]:
import socket
import tkinter as tk
from tkinter import filedialog
import numpy as np
from PIL import Image, ImageTk
import pandas as pd
import re
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

In [2]:
dataset=pd.read_excel('res.xlsx')

In [3]:
def communicate_with_cpp(image_path):
    try:
        # 创建一个套接字对象
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        # 连接到 C++ 服务器
        server_address = ('127.0.0.1', 9898)
        client_socket.connect(server_address)
        # 发送数据到服务器
        message_to_send = image_path
        client_socket.sendall(message_to_send.encode('utf-8'))
        # 接收从服务器返回的数据
        data_received = client_socket.recv(1024).decode('utf-8')
        return data_received 
    except Exception as e:
        print(f"Error communicating with C++: {e}")
        return None
    finally:
        # 关闭连接
        client_socket.close()

In [4]:
def process(data_received):
    pattern = r'(J:\\WorkWork\\bugproject\\data_images\\[^\s]+\.jpg)'
    file_paths = re.findall(r'J:\\WorkWork\\bugproject\\data_images\\[^J]*\.jpg', data_received)
    file_names = [re.search(r'\\([^\\]*)$', path).group(1) for path in file_paths]
    result = dataset[dataset['IMG'].isin(file_names)]
    file_mapping = {name: path for name, path in zip(file_names, file_paths)}
    result=result.sort_values('Dealtime')
    return_paths = [file_mapping[name] for name in result['IMG'].tolist()]
    result['Price'] = result['Price'].str.replace(',', '')
    result['Price'] = result['Price'].astype(float)
    return result['Dealtime'].tolist(),result['Price'].tolist(),return_paths

In [ ]:
labels = []
def browse_file():
    file_path = filedialog.askopenfilename()
    entry_path.delete(0, tk.END)
    entry_path.insert(0, file_path)
def on_submit():
    global dealtime, price,paths,PriceImg,index
    jpg = entry_path.get()
    data=communicate_with_cpp(jpg)
    dealtime,price,paths=process(data)
    index=len(dealtime)
    
def display_data():
    # 显示五张照片
    chart_window = tk.Toplevel(root)
    chart_window.title('Matplotlib Chart')
    images_window = tk.Toplevel(root)
    images_window.title("Images")
    for i in range(index):
        original_img = Image.open(paths[i])
        resized_img = original_img.resize((100, 300), Image.LANCZOS)
        img = ImageTk.PhotoImage(resized_img)
        label = tk.Label(images_window, image=img)
        label.image = img
        label.grid(row=0, column=i)  # 使用grid布局
        labels.append(label)
    fig = Figure(figsize=(6, 5), dpi=100)
    ax = fig.add_subplot(111)
    df = pd.DataFrame({'Time': dealtime, 'Price': price})
    df['Price'] = df['Price'].astype(float)
    # 设置日期为 x 轴
    ax.plot(df['Time'], df['Price'], marker='o', linestyle='-', color='b')
    ax.set_xticks(range(len(df['Time'])))
    ax.set_xticklabels(df['Time'], rotation=45, ha='right', fontsize=7)
    ax.set_title('Price Trend Over Time')
    ax.set_xlabel('Time')
    ax.set_ylabel('Price')
    ax.grid(True)
    for i, (T, P) in enumerate(zip(df['Time'], df['Price'])):
        ax.text(T, P, f'{P}', ha='left', va='bottom')
    fig.tight_layout()
    canvas = FigureCanvasTkAgg(fig, master=chart_window)

    canvas_widget = canvas.get_tk_widget()
    canvas_widget.pack()
# 销毁之前的Label和图片
def destroy_previous_image():
    global labels
    for label in labels:
        label.destroy()
    labels = []
    
# 创建主窗口
root = tk.Tk()
root.title("GUI 示例")
# 创建并放置 GUI 元素
label_path = tk.Label(root, text="JPG 文件路径:")
label_path.pack()
entry_path = tk.Entry(root, width=50)
entry_path.pack()
button_browse = tk.Button(root, text="浏览", command=browse_file)
button_browse.pack()
button_submit = tk.Button(root, text="提交", command=on_submit)
button_submit.pack()
button_display_next = tk.Button(root, text="显示数据", command=display_data)
button_display_next.pack()


# 开始 GUI 主循环
if __name__ == '__main__':
    root.mainloop()

In [116]:
for i, (time, price) in enumerate(zip(df['Time'], df['Price'])):
    ax.text(time, price, f'{price}', ha='left', va='bottom')